# Regression Week 5: LASSO (coordinate descent)

In this notebook, you will implement your very own LASSO solver via coordinate descent. You will:
* Write a function to normalize features
* Implement coordinate descent for LASSO
* Explore effects of L1 penalty

# Fire up graphlab create

Make sure you have the latest version of graphlab (>= 1.7)

In [1]:
import graphlab

# Load in house sales data

Dataset is from house sales in King County, the region where the city of Seattle, WA is located.

In [2]:
sales = graphlab.SFrame('kc_house_data.gl/')
# In the dataset, 'floors' was defined with type string, 
# so we'll convert them to int, before using it below
sales['floors'] = sales['floors'].astype(int) 

This non-commercial license of GraphLab Create for academic use is assigned to sharecj@126.com and will expire on May 02, 2018.


[INFO] graphlab.cython.cy_server: GraphLab Create v2.1 started. Logging: /tmp/graphlab_server_1494644133.log


If we want to do any "feature engineering" like creating new features or adjusting existing ones we should do this directly using the SFrames as seen in the first notebook of Week 2. For this notebook, however, we will work with the existing features.

# Import useful functions from previous notebook

As in Week 2, we convert the SFrame into a 2D Numpy array. Copy and paste `get_num_data()` from the second notebook of Week 2.

In [3]:
import numpy as np # note this allows us to refer to numpy as np instead 

In [4]:
def get_numpy_data(data_sframe, features, output):
    data_sframe['constant'] = 1 # this is how you add a constant column to an SFrame
    # add the column 'constant' to the front of the features list so that we can extract it along with the others:
    features = ['constant'] + features # this is how you combine two lists
    # select the columns of data_SFrame given by the features list into the SFrame features_sframe (now including constant):
    features_sframe = data_sframe[features]
    # the following line will convert the features_SFrame into a numpy matrix:
    feature_matrix = features_sframe.to_numpy()
    # assign the column of data_sframe associated with the output to the SArray output_sarray
    output_array = data_sframe[output]
    # the following will convert the SArray into a numpy array by first converting it to a list
    output_array = output_array.to_numpy()
    return(feature_matrix, output_array)

Also, copy and paste the `predict_output()` function to compute the predictions for an entire matrix of features given the matrix and the weights:

In [5]:
def predict_output(feature_matrix, weights):
    # assume feature_matrix is a numpy matrix containing the features as columns and weights is a corresponding numpy array
    # create the predictions vector by using np.dot()
    predictions = np.dot(feature_matrix, weights)

    return(predictions)

# Normalize features
In the house dataset, features vary wildly in their relative magnitude: `sqft_living` is very large overall compared to `bedrooms`, for instance. As a result, weight for `sqft_living` would be much smaller than weight for `bedrooms`. This is problematic because "small" weights are dropped first as `l1_penalty` goes up. 

To give equal considerations for all features, we need to **normalize features** as discussed in the lectures: we divide each feature by its 2-norm so that the transformed feature has norm 1.

Let's see how we can do this normalization easily with Numpy: let us first consider a small matrix.

In [6]:
X = np.array([[3.,5.,8.],[4.,12.,15.]])
print X

[[  3.   5.   8.]
 [  4.  12.  15.]]


Numpy provides a shorthand for computing 2-norms of each column:

In [7]:
norms = np.linalg.norm(X, axis=0) # gives [norm(X[:,0]), norm(X[:,1]), norm(X[:,2])]
print norms

[  5.  13.  17.]


To normalize, apply element-wise division:

In [8]:
print X / norms # gives [X[:,0]/norm(X[:,0]), X[:,1]/norm(X[:,1]), X[:,2]/norm(X[:,2])]

[[ 0.6         0.38461538  0.47058824]
 [ 0.8         0.92307692  0.88235294]]


Using the shorthand we just covered, write a short function called `normalize_features(feature_matrix)`, which normalizes columns of a given feature matrix. The function should return a pair `(normalized_features, norms)`, where the second item contains the norms of original features. As discussed in the lectures, we will use these norms to normalize the test data in the same way as we normalized the training data. 

In [11]:
def normalize_features(feature_matrix):
    norms = np.linalg.norm(feature_matrix, axis=0)
    normalized_features = feature_matrix / norms
    return normalized_features, norms

To test the function, run the following:

In [12]:
features, norms = normalize_features(np.array([[3.,6.,9.],[4.,8.,12.]]))
print features
# should print
# [[ 0.6  0.6  0.6]
#  [ 0.8  0.8  0.8]]
print norms
# should print
# [5.  10.  15.]

[[ 0.6  0.6  0.6]
 [ 0.8  0.8  0.8]]
[  5.  10.  15.]


# Implementing Coordinate Descent with normalized features

We seek to obtain a sparse set of weights by minimizing the LASSO cost function
```
SUM[ (prediction - output)^2 ] + lambda*( |w[1]| + ... + |w[k]|).
```
(By convention, we do not include `w[0]` in the L1 penalty term. We never want to push the intercept to zero.)

The absolute value sign makes the cost function non-differentiable, so simple gradient descent is not viable (you would need to implement a method called subgradient descent). Instead, we will use **coordinate descent**: at each iteration, we will fix all weights but weight `i` and find the value of weight `i` that minimizes the objective. That is, we look for
```
argmin_{w[i]} [ SUM[ (prediction - output)^2 ] + lambda*( |w[1]| + ... + |w[k]|) ]
```
where all weights other than `w[i]` are held to be constant. We will optimize one `w[i]` at a time, circling through the weights multiple times.  
  1. Pick a coordinate `i`
  2. Compute `w[i]` that minimizes the cost function `SUM[ (prediction - output)^2 ] + lambda*( |w[1]| + ... + |w[k]|)`
  3. Repeat Steps 1 and 2 for all coordinates, multiple times

For this notebook, we use **cyclical coordinate descent with normalized features**, where we cycle through coordinates 0 to (d-1) in order, and assume the features were normalized as discussed above. The formula for optimizing each coordinate is as follows:
```
       ┌ (ro[i] + lambda/2)     if ro[i] < -lambda/2
w[i] = ├ 0                      if -lambda/2 <= ro[i] <= lambda/2
       └ (ro[i] - lambda/2)     if ro[i] > lambda/2
```
where
```
ro[i] = SUM[ [feature_i]*(output - prediction + w[i]*[feature_i]) ].
```

Note that we do not regularize the weight of the constant feature (intercept) `w[0]`, so, for this weight, the update is simply:
```
w[0] = ro[i]
```

## Effect of L1 penalty

Let us consider a simple model with 2 features:

In [169]:
simple_features = ['sqft_living', 'bedrooms']
my_output = 'price'
(simple_feature_matrix, output) = get_numpy_data(sales, simple_features, my_output)

Don't forget to normalize features:

In [170]:
simple_feature_matrix, norms = normalize_features(simple_feature_matrix)

We assign some random set of initial weights and inspect the values of `ro[i]`:

In [171]:
weights = np.array([1., 4., 1.])

Use `predict_output()` to make predictions on this data.

In [174]:
prediction = predict_output(simple_feature_matrix, weights)

Compute the values of `ro[i]` for each feature in this simple model, using the formula given above, using the formula:
```
ro[i] = SUM[ [feature_i]*(output - prediction + w[i]*[feature_i]) ]
```

*Hint: You can get a Numpy vector for feature_i using:*
```
simple_feature_matrix[:,i]
```

In [175]:
w=weights[1]
i=1
feature_i = simple_feature_matrix[:, i]
ro = np.dot(feature_i, (output - prediction + w * feature_i))
print ro

87939470.773


***QUIZ QUESTION***

Recall that, whenever `ro[i]` falls between `-l1_penalty/2` and `l1_penalty/2`, the corresponding weight `w[i]` is sent to zero. Now suppose we were to take one step of coordinate descent on either feature 1 or feature 2. What range of values of `l1_penalty` **would not** set `w[1]` zero, but **would** set `w[2]` to zero, if we were to take a step in that coordinate? 

***QUIZ QUESTION***

What range of values of `l1_penalty` would set **both** `w[1]` and `w[2]` to zero, if we were to take a step in that coordinate? 

In [178]:
w=weights[2]
i=2
feature_i = simple_feature_matrix[:, i]
ro = np.dot(feature_i, (output - prediction + w * feature_i))
print ro

80966698.676


So we can say that `ro[i]` quantifies the significance of the i-th feature: the larger `ro[i]` is, the more likely it is for the i-th feature to be retained.

## Single Coordinate Descent Step

Using the formula above, implement coordinate descent that minimizes the cost function over a single feature i. Note that the intercept (weight 0) is not regularized. The function should accept feature matrix, output, current weights, l1 penalty, and index of feature to optimize over. The function should return new weight for feature i.

In [127]:
def lasso_coordinate_descent_step(i, feature_matrix, output, weights, l1_penalty):
    # compute prediction
    prediction = predict_output(feature_matrix, weights)
    # compute ro[i] = SUM[ [feature_i]*(output - prediction + weight[i]*[feature_i]) ]
    feature_i = feature_matrix[:, i]
    ro_i = np.dot(feature_i, (output - prediction + weights[i] * feature_i))

    if i == 0: # intercept -- do not regularize
        new_weight_i = ro_i 
    elif ro_i < -l1_penalty/2.:
        new_weight_i = ro_i + l1_penalty/2
    elif ro_i > l1_penalty/2.:
        new_weight_i = ro_i - l1_penalty/2
    else:
        new_weight_i = 0.
    
    return new_weight_i

To test the function, run the following cell:

In [128]:
# should print 0.425558846691
import math
print lasso_coordinate_descent_step(1, np.array([[3./math.sqrt(13),1./math.sqrt(10)],[2./math.sqrt(13),3./math.sqrt(10)]]), 
                                   np.array([1., 1.]), np.array([1., 4.]), 0.1)

0.425558846691


## Cyclical coordinate descent 

Now that we have a function that optimizes the cost function over a single coordinate, let us implement cyclical coordinate descent where we optimize coordinates 0, 1, ..., (d-1) in order and repeat.

When do we know to stop? Each time we scan all the coordinates (features) once, we measure the change in weight for each coordinate. If no coordinate changes by more than a specified threshold, we stop.

For each iteration:
1. As you loop over features in order and perform coordinate descent, measure how much each coordinate changes.
2. After the loop, if the maximum change across all coordinates is falls below the tolerance, stop. Otherwise, go back to step 1.

Return weights

**IMPORTANT: when computing a new weight for coordinate i, make sure to incorporate the new weights for coordinates 0, 1, ..., i-1. One good way is to update your weights variable in-place. See following pseudocode for illustration.**
```
for i in range(len(weights)):
    old_weights_i = weights[i] # remember old value of weight[i], as it will be overwritten
    # the following line uses new values for weight[0], weight[1], ..., weight[i-1]
    #     and old values for weight[i], ..., weight[d-1]
    weights[i] = lasso_coordinate_descent_step(i, feature_matrix, output, weights, l1_penalty)
    
    # use old_weights_i to compute change in coordinate
    ...
```

In [68]:
def lasso_cyclical_coordinate_descent(feature_matrix, output, initial_weights, l1_penalty, tolerance):
    weights = initial_weights
    while True:
        max_change = 0
        for i in range(len(weights)):
            old_weights_i = weights[i]
            weights[i] = lasso_coordinate_descent_step(i, feature_matrix, output, weights, l1_penalty)
            change = abs(weights[i] - old_weights_i)
            if change > max_change:
                max_change = change
        print max_change
        if max_change < tolerance:
            break
    return weights

Using the following parameters, learn the weights on the sales dataset. 

In [137]:
simple_features = ['sqft_living', 'bedrooms']
my_output = 'price'
initial_weights = np.zeros(3)
l1_penalty = 1e7
tolerance = 1.0

First create a normalized version of the feature matrix, `normalized_simple_feature_matrix`.

In [138]:
(simple_feature_matrix, output) = get_numpy_data(sales, simple_features, my_output)
(normalized_simple_feature_matrix, simple_norms) = normalize_features(simple_feature_matrix) # normalize features

Then, run your implementation of LASSO coordinate descent:

In [139]:
weights = lasso_cyclical_coordinate_descent(normalized_simple_feature_matrix, output,
                                            initial_weights, l1_penalty, tolerance)
print weights

79400304.6581
9138168.3665
8194809.40937
6598905.03827
5522173.1943
4621129.81031
3867108.10625
3236118.81061
2708086.93955
2266213.1094
1896439.06265
1587000.40319
1328052.31095
1111356.328
930018.251241
778268.79269
651280.029038
545011.800818
456083.174344
381664.876995
319389.283634
267275.090398
223664.279321
187169.368346
156629.268444
131072.343461
109685.497423
91788.3058116
76811.3677894
64278.190664
53790.0302253
45013.2046617
37668.4784415
31522.1784089
26378.7594496
22074.5832053
18472.7119036
15458.5516701
12936.2067131
10825.4283904
9059.06209101
7580.91070477
6343.94670621
5308.81596931
4442.58571219
3717.6967377
3111.08663489
2603.45604621
2178.65465674
1823.16737023
1525.68432523
1276.74107066
1068.41745343
894.085638065
748.199241444
626.11687411
523.954474047
438.461734913
366.918697149
307.049212292
256.948526978
215.0226833
179.937806472
150.577667948
126.008172076
105.447638087
88.2419305816
73.8436484337
61.7947089039
51.7117738575
43.2740537673
36.2131017856
30.

***QUIZ QUESTIONS***
1. What is the RSS of the learned model on the normalized dataset? (Hint: use the normalized feature matrix when you make predictions.)
2. Which features had weight zero at convergence?

In [140]:
predictions = np.dot(normalized_simple_feature_matrix, weights)
residuals = predictions - output
RSS = np.dot(residuals, residuals)
print RSS

1.63049248148e+15


# Evaluating LASSO fit with more features

Let us split the sales dataset into training and test sets.

In [164]:
train_data,test_data = sales.random_split(.8,seed=0)

Let us consider the following set of features.

In [165]:
all_features = ['bedrooms',
                'bathrooms',
                'sqft_living',
                'sqft_lot',
                'floors',
                'waterfront', 
                'view', 
                'condition', 
                'grade',
                'sqft_above',
                'sqft_basement',
                'yr_built', 
                'yr_renovated']

def get_non_zero_feature(weights):
    indexes = weights.nonzero()[0]
    features = []
    for _index in indexes:
        _index -= 1
        if _index >= 0:
            features.append(all_features[_index])
    return features

First, create a normalized feature matrix from the TRAINING data with these features.  (Make you store the norms for the normalization, since we'll use them later)

In [166]:
(all_feature_matrix, output) = get_numpy_data(train_data, all_features, my_output)
(normalized_all_feature_matrix, all_norms) = normalize_features(all_feature_matrix) # normalize features

First, learn the weights with `l1_penalty=1e7`, on the training data. Initialize weights to all zeros, and set the `tolerance=1`.  Call resulting weights `weights1e7`, you will need them later.

In [167]:
initial_weights = np.zeros(len(all_features)+1)
l1_penalty = 1e7
tolerance = 1.0
weights1e7 = lasso_cyclical_coordinate_descent(normalized_all_feature_matrix, output,
                                            initial_weights, l1_penalty, tolerance)
print weights1e7
print get_non_zero_feature(weights1e7)

71114625.7528
5024356.36869
5012324.91395
5000676.42086
4991243.17314
4984361.34335
4979504.92921
4973806.56525
2879222.99938
2706150.76622
2532295.64864
2368316.57782
2213124.35067
1552443.60013
507838.033159
438814.370591
371479.622608
313415.105581
264244.832139
222753.8608
187770.667306
158280.093357
133420.901606
112465.988296
94802.2134519
79912.6874505
67361.6926196
56781.942492
47863.835752
40346.3966195
34009.6378518
28668.1231465
24165.5406129
20370.1285268
17170.8195088
14473.990295
12200.7219837
10284.4905855
8669.22029243
7307.64249855
6159.91255101
5192.44375234
4376.92449333
3689.48975359
3110.02272543
2621.56612397
2209.82595598
1862.75322644
1570.19134161
1323.57889082
1115.69910862
940.468686543
792.759753536
668.24981609
563.295266584
474.824758224
400.249326415
337.38662637
284.397069871
239.729992338
202.078274757
170.340092748
143.586673204
121.035115022
102.025478713
86.0014740676
72.4941812083
61.1083282791
51.5107243322
43.4205090702
36.6009337157
30.8524330594

***QUIZ QUESTION***

What features had non-zero weight in this case?

Next, learn the weights with `l1_penalty=1e8`, on the training data. Initialize weights to all zeros, and set the `tolerance=1`.  Call resulting weights `weights1e8`, you will need them later.

In [168]:
initial_weights = np.zeros(len(all_features)+1)
l1_penalty = 1e8
tolerance = 1.0
weights1e8 = lasso_cyclical_coordinate_descent(normalized_all_feature_matrix, output,
                                            initial_weights, l1_penalty, tolerance)
print weights1e8
print get_non_zero_feature(weights1e8)

71114625.7528
0
[ 71114625.75280938         0.                 0.                 0.
         0.                 0.                 0.                 0.
         0.                 0.                 0.                 0.
         0.                 0.        ]
[]


***QUIZ QUESTION***

What features had non-zero weight in this case?

Finally, learn the weights with `l1_penalty=1e4`, on the training data. Initialize weights to all zeros, and set the `tolerance=5e5`.  Call resulting weights `weights1e4`, you will need them later.  (This case will take quite a bit longer to converge than the others above.)

In [144]:
initial_weights = np.zeros(len(all_features)+1)
l1_penalty = 1e4
tolerance = 5e5
weights1e4 = lasso_cyclical_coordinate_descent(normalized_all_feature_matrix, output,
                                            initial_weights, l1_penalty, tolerance)
print weights1e4
print get_non_zero_feature(weights1e4)

71114625.7528
8464015.94738
6818451.35869
5570165.21901
4559838.46759
3732056.69715
3053211.61353
2497517.43558
2043868.97325
1674717.70906
1375420.75692
1296621.8143
1257340.9361
1228271.24819
1206854.36471
1191059.40598
1179030.77017
1169777.43345
1162340.67589
1155980.52901
1150139.67966
1144406.32752
1138483.91805
1132166.57169
1125319.16778
1117861.24888
1109754.06282
1100990.17907
1091585.21244
1081571.26761
1070991.78405
1059897.51701
1048343.43631
1036386.3636
1024083.20068
1011489.62847
998659.178392
985642.596017
972487.432403
959237.810727
945934.326266
932614.046118
919310.581988
906054.214956
892872.055697
879109.302243
866038.266742
853191.44943
840521.781012
828028.403962
815717.920819
803598.206378
791676.769693
779960.222854
768454.139375
757163.05916
746090.546302
735239.263812
724611.051693
714002.910141
703393.002342
693038.299304
682960.589206
673155.135299
663513.600594
654148.681361
645045.446052
636191.878813
627577.229773
619191.96692
611027.657019
603076.79613

***QUIZ QUESTION***

What features had non-zero weight in this case?

## Rescaling learned weights

Recall that we normalized our feature matrix, before learning the weights.  To use these weights on a test set, we must normalize the test data in the same way.

Alternatively, we can rescale the learned weights to include the normalization, so we never have to worry about normalizing the test data: 

In this case, we must scale the resulting weights so that we can make predictions with *original* features:
 1. Store the norms of the original features to a vector called `norms`:
```
features, norms = normalize_features(features)
```
 2. Run Lasso on the normalized features and obtain a `weights` vector
 3. Compute the weights for the original features by performing element-wise division, i.e.
```
weights_normalized = weights / norms
```
Now, we can apply `weights_normalized` to the test data, without normalizing it!

Create a normalized version of each of the weights learned above. (`weights1e4`, `weights1e7`, `weights1e8`).

In [145]:
weights1e4_normalized = weights1e4 / all_norms
weights1e7_normalized = weights1e7 / all_norms
weights1e8_normalized = weights1e8 / all_norms
print weights1e7
print weights1e7_normalized
print weights1e7_normalized[3]

[ 24429600.60933314         0.                 0.          48389174.35227978
         0.                 0.           3317511.16271981   7329961.9848964
         0.                 0.                 0.                 0.
         0.                 0.        ]
[  1.85285533e+05   0.00000000e+00   0.00000000e+00   1.61317456e+02
   0.00000000e+00   0.00000000e+00   2.87664700e+05   6.91937057e+04
   0.00000000e+00   0.00000000e+00   0.00000000e+00   0.00000000e+00
   0.00000000e+00   0.00000000e+00]
161.317456248


To check your results, if you call `normalized_weights1e7` the normalized version of `weights1e7`, then:
```
print normalized_weights1e7[3]
```
should return 161.31745624837794.

## Evaluating each of the learned models on the test data

Let's now evaluate the three models on the test data:

In [146]:
(test_feature_matrix, test_output) = get_numpy_data(test_data, all_features, 'price')

Compute the RSS of each of the three normalized weights on the (unnormalized) `test_feature_matrix`:

In [147]:
prediction = np.dot(test_feature_matrix, weights1e4_normalized)
residuals = prediction - test_output
RSS = np.dot(residuals, residuals)
print RSS

2.2778100476e+14


In [148]:
prediction = np.dot(test_feature_matrix, weights1e7_normalized)
residuals = prediction - test_output
RSS = np.dot(residuals, residuals)
print RSS

2.75962079909e+14


In [149]:
prediction = np.dot(test_feature_matrix, weights1e8_normalized)
residuals = prediction - test_output
RSS = np.dot(residuals, residuals)
print RSS

5.37166150034e+14


***QUIZ QUESTION***

Which model performed best on the test data?